In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import nltk
import jieba
import re
from nltk.corpus import stopwords
import gensim
from gensim.models import word2vec
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers.embeddings import Embedding
from keras.layers.core import Flatten

In [2]:
df_original = pd.read_csv('steam_games.csv')
stopwords =  stopwords.words('english')
user = pd.read_csv('steam_cleaned_less.csv')

In [3]:
user.head()

,Unnamed: 0,review_id,review,app_id,name,url,author.steamid,recommended,votes_helpful,votes_funny,...,author.playtime_at_review,author.last_played,written_during_early_access,developer,desc_snippet,game_description,genre,popular_tags,game_details,languages
0,0,85174926,The game is enjoyable enough but...\n-Combat h...,292030,The Witcher® 3: Wild Hunt,https://store.steampowered.com/app/292030/The_...,76561198020027165,True,0,0,...,398.0,1.611370e+09,False,CD PROJEKT RED,As war rages on throughout the Northern Realms...,About This Game The Witcher: Wild Hunt is a s...,RPG,"Open World,RPG,Story Rich,Atmospheric,Mature,F...","Single-player,Steam Achievements,Full controll...","English,French,Italian,German,Spanish - Spain,..."
1,1,85165539,like dark souls but with witches,292030,The Witcher® 3: Wild Hunt,https://store.steampowered.com/app/292030/The_...,76561198983515611,True,0,0,...,325.0,1.610922e+09,False,CD PROJEKT RED,As war rages on throughout the Northern Realms...,About This Game The Witcher: Wild Hunt is a s...,RPG,"Open World,RPG,Story Rich,Atmospheric,Mature,F...","Single-player,Steam Achievements,Full controll...","English,French,Italian,German,Spanish - Spain,..."
2,3,85155206,They certainly dumbed down the series since th...,292030,The Witcher® 3: Wild Hunt,https://store.steampowered.com/app/292030/The_...,76561198010955177,False,0,0,...,8966.0,1.514417e+09,False,CD PROJEKT RED,As war rages on throughout the Northern Realms...,About This Game The Witcher: Wild Hunt is a s...,RPG,"Open World,RPG,Story Rich,Atmospheric,Mature,F...","Single-player,Steam Achievements,Full controll...","English,French,Italian,German,Spanish - Spain,..."
3,14,85049145,⠀,292030,The Witcher® 3: Wild Hunt,https://store.steampowered.com/app/292030/The_...,76561197969538443,True,0,0,...,7072.0,1.529545e+09,False,CD PROJEKT RED,As war rages on throughout the Northern Realms...,About This Game The Witcher: Wild Hunt is a s...,RPG,"Open World,RPG,Story Rich,Atmospheric,Mature,F...","Single-player,Steam Achievements,Full controll...","English,French,Italian,German,Spanish - Spain,..."
4,25,84841853,i mean the reviews say it all,292030,The Witcher® 3: Wild Hunt,https://store.steampowered.com/app/292030/The_...,76561198173380217,True,1,0,...,231.0,1.591251e+09,False,CD PROJEKT RED,As war rages on throughout the Northern Realms...,About This Game The Witcher: Wild Hunt is a s...,RPG,"Open World,RPG,Story Rich,Atmospheric,Mature,F...","Single-player,Steam Achievements,Full controll...","English,French,Italian,German,Spanish - Spain,..."


In [4]:
gameList = user.name.unique()
idList = user.app_id.unique()

## 1.Dataset cleaning and preprocessing 

In [5]:
gameSet = pd.DataFrame()
for i in range(len(gameList)):
    for j in range(len(df_original)):
        if gameList[i] == df_original.name.values[j]:
            gameSet = gameSet.append(df_original.iloc[j])
    

In [6]:
gameSet = gameSet.reset_index(drop=True) 
gameSet.head()

,achievements,all_reviews,desc_snippet,developer,discount_price,game_description,game_details,genre,languages,mature_content,minimum_requirements,name,original_price,popular_tags,publisher,recent_reviews,recommended_requirements,release_date,types,url
0,78.0,"Overwhelmingly Positive,(189,538),- 97% of the...",As war rages on throughout the Northern Realms...,CD PROJEKT RED,$11.99,About This Game The Witcher: Wild Hunt is a s...,"Single-player,Steam Achievements,Full controll...",RPG,"English,French,Italian,German,Spanish - Spain,...",Mature Content Description The developers de...,"Minimum:,OS:,64-bit Windows 7, 64-bit Windows ...",The Witcher® 3: Wild Hunt,$39.99,"Open World,RPG,Story Rich,Atmospheric,Mature,F...","CD PROJEKT RED,CD PROJEKT RED","Overwhelmingly Positive,(2,386),- 96% of the 2...","Recommended:,OS:,64-bit Windows 7, 64-bit Wind...","May 18, 2015",app,https://store.steampowered.com/app/292030/The_...
1,NaN,"Overwhelmingly Positive,(22,515),- 95% of the ...",Named Game of the Year by over 50 publications...,Valve,$65.70,About This Game Named Game of the Year by ove...,"Single-player,Multi-player,Online Multi-Player...",Action,"English,French,German,Italian,Spanish - Spain,...",NaN,NaN,Half-Life,$9.99,"FPS,Classic,Action,Sci-fi,Singleplayer,Masterp...","Valve,Valve","Overwhelmingly Positive,(235),- 96% of the 235...",NaN,"Nov 8, 1998",app,https://store.steampowered.com/app/70/HalfLife/
2,147.0,"Overwhelmingly Positive,(45,829),- 95% of the ...","Just updated to include player stats, achievem...",Valve,$65.70,About This Game THE NEXT INSTALLMENT OF THE W...,"Multi-player,Cross-Platform Multiplayer,Steam ...",Action,"English,French,German,Italian,Japanese,Korean,...",Mature Content Description The developers de...,NaN,Counter-Strike: Source,$9.99,"Shooter,Action,FPS,Multiplayer,Team-Based,Firs...","Valve,Valve","Very Positive,(439),- 94% of the 439 user revi...",NaN,"Nov 1, 2004",app,https://store.steampowered.com/app/240/Counter...
3,22.0,"Overwhelmingly Positive,(10,894),- 95% of the ...",Half-Life® 2: Episode Two is the second in a t...,Valve,$65.70,About This Game Half-Life® 2: Episode Two is...,"Single-player,Steam Achievements,Captions avai...",Action,"English,French,German,Russian,Danish,Dutch,Fin...",NaN,NaN,Half-Life 2: Episode Two,$7.99,"FPS,Action,Sci-fi,Singleplayer,Shooter,Story R...","Valve,Valve","Overwhelmingly Positive,(143),- 97% of the 143...",NaN,"Oct 10, 2007",app,https://store.steampowered.com/app/420/HalfLif...
4,51.0,"Overwhelmingly Positive,(104,354),- 98% of the...","The ""Perpetual Testing Initiative"" has been ex...",Valve,$14.98,About This Game Portal 2 draws from the award...,"Single-player,Co-op,Steam Achievements,Full co...","Action,Adventure","English,French,German,Spanish - Spain,Czech,Da...",NaN,NaN,Portal 2,$9.99,"Puzzle,Co-op,First-Person,Sci-fi,Comedy,Single...","Valve,Valve","Overwhelmingly Positive,(899),- 98% of the 899...",NaN,"Apr 18, 2011",app,https://store.steampowered.com/app/620/Portal_2/


In [7]:
temp = gameSet['all_reviews'].values
temp[34] = '90%'
gameSet['all_reviews'] = temp

In [8]:
temp = gameSet['publisher'].values
temp[21] = 'Team meat'
gameSet['publisher'] = temp

In [9]:
gameSet = gameSet[gameSet.genre.notnull()]
gameSet = gameSet[gameSet.game_details.notnull()]
gameSet = gameSet[gameSet.all_reviews.notnull()]
gameSet = gameSet[gameSet.publisher.notnull()]
gameSet.shape

(136, 20)

In [10]:
gameSet = gameSet[['name','desc_snippet','genre','popular_tags','all_reviews','developer','publisher']]
gameSet.head()

,name,desc_snippet,genre,popular_tags,all_reviews,developer,publisher
0,The Witcher® 3: Wild Hunt,As war rages on throughout the Northern Realms...,RPG,"Open World,RPG,Story Rich,Atmospheric,Mature,F...","Overwhelmingly Positive,(189,538),- 97% of the...",CD PROJEKT RED,"CD PROJEKT RED,CD PROJEKT RED"
1,Half-Life,Named Game of the Year by over 50 publications...,Action,"FPS,Classic,Action,Sci-fi,Singleplayer,Masterp...","Overwhelmingly Positive,(22,515),- 95% of the ...",Valve,"Valve,Valve"
2,Counter-Strike: Source,"Just updated to include player stats, achievem...",Action,"Shooter,Action,FPS,Multiplayer,Team-Based,Firs...","Overwhelmingly Positive,(45,829),- 95% of the ...",Valve,"Valve,Valve"
3,Half-Life 2: Episode Two,Half-Life® 2: Episode Two is the second in a t...,Action,"FPS,Action,Sci-fi,Singleplayer,Shooter,Story R...","Overwhelmingly Positive,(10,894),- 95% of the ...",Valve,"Valve,Valve"
4,Portal 2,"The ""Perpetual Testing Initiative"" has been ex...","Action,Adventure","Puzzle,Co-op,First-Person,Sci-fi,Comedy,Single...","Overwhelmingly Positive,(104,354),- 98% of the...",Valve,"Valve,Valve"


In [11]:
names = gameSet.name.values
idList = []
for i in range(len(names)):
    idList.append(user[user['name'] == names[i]].app_id.values[0])

In [12]:
gameSet['app_id'] = idList

In [13]:
def lowerCase(string):
    return string.lower()

def seg_depart(sentence):
    return nltk.word_tokenize(sentence)

def removeUrl(string):
    url = re.findall('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', string)    
    if len(url) > 0:
        for i in range(len(url)):
            string = string.replace(url[i],'')
    return string
        
def remove_digits(input_str):
    punc = u'0123456789.'
    output_str = re.sub(r'[{}]+'.format(punc), '', input_str)
    return output_str

def move_stopwords(sentence_list, stopwords_list):
    out_list = []
    for word in sentence_list:
        if word not in stopwords_list:
            if not remove_digits(word):
                continue
            if word != '\t':
                out_list.append(word)
    return out_list

def cleanlines(line): 
    p4=re.compile(r'[\s+\.\!\/_,$%^*(+\"\')]+|[+——()?【】“”！，。？、~@#￥%……&*（）0-9 , : ; \-\ \[\ \]\ ]')
    line=p4.sub(r' ',str(line))    
    return line


def removeDuplicate(l1):
    return list(set(l1))

In [14]:
def isLengthOne(string):
    return len(string) == 1

def processInDetail(l1):
    #rstr = r"[\=\(\)\,\/\\\:\*\?\"\<\>\|\' ']" 
    rstr = r"[\↦\→\=\(\)\,\/\\\:\*\?\"\<\>\|\' ']" 
    i = 0 
    remove_list = ['xa','xd']
    for word in l1:
        i = i + 1
        if isLengthOne(word):
            remove_list.append(word)
        if len(re.findall(rstr, word)) > 0:
            remove_list.append(word)
        #remove xa,xd
  
    return [i for i in l1 if i not in remove_list]

In [15]:
def process(string):
    sentence = removeUrl(string)
    sentence = cleanlines(sentence)
    sentence = lowerCase(sentence)
    #print(sentence)
    
    sentence_depart = seg_depart(sentence)
    
    #print(sentence_depart)
    sentence_depart = move_stopwords(sentence_depart, stopwords)
    #print(sentence_depart)
    return removeDuplicate(processInDetail(sentence_depart))

In [16]:
processedList = []
for i in range(len(gameSet)):
    processedList.append(process(gameSet.iloc[i]['desc_snippet']))    

In [17]:
len(processedList)

136

## 2.Feature engineering

### Extract rating

In [18]:
rating = []
for i in gameSet.all_reviews.values:
    try:
        string = re.findall('\d+%',i)[0].replace('%','')
        rating.append(int(string))
    except:
        rating.append(0)

### Pubilsher

In [19]:
len(list(set(gameSet.publisher.values)))

101

In [20]:
le = preprocessing.LabelEncoder()
le.fit(list(set(gameSet.publisher.values)))

LabelEncoder()

In [21]:
publisher = le.transform(gameSet.publisher.values)

In [22]:
publisher = np.array(publisher)
input_dim = len(list(set(gameSet.publisher.values)))
print(input_dim)

embedding_size = 30
model = Sequential()
model.add(Embedding(input_dim = input_dim, output_dim = embedding_size, input_length = 1, name="embedding"))
model.add(Flatten())
model.add(Dense(50, activation="relu"))
model.add(Dense(15, activation="relu"))
model.add(Dense(1))
model.compile(loss = "mse", optimizer = "adam", metrics=["accuracy"])
model.fit(np.array(publisher), np.array(rating), epochs = 20, batch_size = 4)

101
Epoch 1/20
34/34 [==============================] - 1s 3ms/step - loss: 7375.4419 - accuracy: 0.0000e+00
Epoch 2/20
34/34 [==============================] - 0s 2ms/step - loss: 7313.9012 - accuracy: 0.0000e+00
Epoch 3/20
34/34 [==============================] - 0s 2ms/step - loss: 7461.2668 - accuracy: 0.0000e+00
Epoch 4/20
34/34 [==============================] - 0s 2ms/step - loss: 6727.0183 - accuracy: 0.0000e+00
Epoch 5/20
34/34 [==============================] - 0s 2ms/step - loss: 6368.9345 - accuracy: 0.0000e+00
Epoch 6/20
34/34 [==============================] - 0s 2ms/step - loss: 4422.4385 - accuracy: 0.0000e+00
Epoch 7/20
34/34 [==============================] - 0s 2ms/step - loss: 2108.1221 - accuracy: 0.0000e+00
Epoch 8/20
34/34 [==============================] - 0s 2ms/step - loss: 720.7311 - accuracy: 0.0000e+00
Epoch 9/20
34/34 [==============================] - 0s 2ms/step - loss: 324.9362 - accuracy: 0.0000e+00
Epoch 10/20
34/34 [==============================] - 

In [23]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1, 30)             3030      
_________________________________________________________________
flatten (Flatten)            (None, 30)                0         
_________________________________________________________________
dense (Dense)                (None, 50)                1550      
_________________________________________________________________
dense_1 (Dense)              (None, 15)                765       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 16        
Total params: 5,361
Trainable params: 5,361
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
embeddings = model.layers[0].get_weights()[0]

In [25]:
publisher_embedding = []
for i in range(len(publisher)):
    publisher_embedding.append(embeddings[publisher[i]])

### Popular tags

In [26]:
tagList = []
for tag in gameSet.popular_tags:
    tagList.append([i.lower() for i in tag.split(',')])

In [27]:
tag = []
for i in tagList:
    for j in i:
        tag.append(j)

In [28]:
len(list(set(tag)))

258

In [29]:
tag_corpus = sorted(list(set(tag)))

In [30]:
tag_bow = []
for l in tagList:
    temp = np.zeros(len(tag_corpus))
    for tag in l:
        for j in range(len(tag_corpus)):
            if tag == tag_corpus[j]:
                temp[j] = temp[j] + 1
    tag_bow.append(temp)    

### Genre

In [31]:
genreList = []
for genre in gameSet.genre:
    genreList.append([i.lower() for i in genre.split(',')])

In [32]:
genre = []
for i in genreList:
    for j in i:
        genre.append(j)

In [33]:
genre_corpus = sorted(list(set(genre)))
genre_corpus

['action',
 'adventure',
 'animation & modeling',
 'casual',
 'design & illustration',
 'early access',
 'free to play',
 'indie',
 'massively multiplayer',
 'racing',
 'rpg',
 'simulation',
 'sports',
 'strategy',
 'utilities']

In [34]:
genre_bow = []
for l in genreList:
    temp = np.zeros(len(genre_corpus))
    for genre in l:
        for j in range(len(genre_corpus)):
            if genre == genre_corpus[j]:
                temp[j] = temp[j] + 1
    genre_bow.append(temp)    

In [35]:
len(genreList)

136

### Word2vec

In [36]:
df_description = df_original[df_original.game_description.notnull()]

In [37]:
w2v_list = []
for i in range(len(df_description)):
    w2v_list.append(process(df_description.iloc[i]['game_description']))


In [38]:
from gensim.models import word2vec 
w2v_model = word2vec.Word2Vec(w2v_list, size=250, window=5, min_count=2, workers=4)

In [39]:
w2v_model.wv.most_similar('sandbox')

[('devices', 0.9897649884223938),
 ('break', 0.9800118207931519),
 ('causes', 0.9788855910301208),
 ('cannons', 0.9785929918289185),
 ('powers', 0.9772711992263794),
 ('assume', 0.9771981239318848),
 ('places', 0.9767069220542908),
 ('forward', 0.9765629768371582),
 ('patience', 0.9754320383071899),
 ('worker', 0.9733378887176514)]

### Vectorize the comments

In [40]:
def get_contentVector(cutWords, word2vec_model):
    vector_list = [word2vec_model.wv[k] for k in cutWords if k in word2vec_model]
    contentVector = np.array(vector_list).mean(axis=0)
    return contentVector

In [41]:
import time
startTime = time.time()
contentVector_list = []
for i in range(len(processedList)):
    cutWords = processedList[i]
    if (i+1) % 100000 == 0:
        usedTime = time.time() - startTime
        print('前%d篇文章内容表示成向量共花费%.2f秒' %(i+1, usedTime))
    if len(cutWords) != 0:
        contentVector_list.append(get_contentVector(cutWords, w2v_model))
    else:
        contentVector_list.append(get_contentVector(cutWords, w2v_model))
X = np.array(contentVector_list)

<ipython-input-40-fa8f9cd7daba>:2: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  vector_list = [word2vec_model.wv[k] for k in cutWords if k in word2vec_model]


In [42]:
rl = []
for i in range(len(X)):
    if np.isnan(contentVector_list[i]).any():
        rl.append(i)        

In [43]:
print('length of genre_bow is ',len(genre_bow))
print('length of embeddings is ',len(publisher_embedding))
print('length of X is ',len(X))

length of genre_bow is  136
length of embeddings is  136
length of X is  136


In [44]:
combined = []
total_size = len(genre_bow[0]) + len(publisher_embedding[0]) + len(X[0])
for i in range(len(X)):
    try:
        combined.append(list(genre_bow[i]) + list(publisher_embedding[i]) + list(X[i]))
    except:        
        combined.append(total_size * [0])

### Compute consine similarities

In [45]:
def cossim(x,y):
    d1=np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))
    return d1

In [46]:
temp = []
for j in range(len(combined)):
    temp.append(cossim(combined[104],combined[j]))
temp[104] = 0


In [47]:
np.array(temp).argsort()[-5:][::-1]

array([132,  91,  94, 111, 106], dtype=int64)

In [48]:
np.sort(np.array(temp))[-5:][::-1]

array([0.72667899, 0.70443735, 0.65757041, 0.65471731, 0.63630754])

### Store the similarities in the matrix

In [49]:
sim_result = [[0]*136 for i in range(136)]

In [50]:
for i in range(len(combined)):
    for j in range(len(combined)):
        if i == j :
            sim_result[i][j] = 0
        else:
            sim_result[i][j] = cossim(combined[i],combined[j])


## 3.Recommend part

In [51]:
df_re=pd.read_csv('steam_cleaned_less.csv',index_col=0)

In [52]:
size_dict = df_re.groupby(['author.steamid']).size().to_dict()


In [53]:
train_df=df_re.groupby(['author.steamid']).apply(lambda x : x.sort_values(by = 'datetime_created', ascending = True).head(round(len(x)*0.8))).reset_index(drop = True)
train_df.head()

,review_id,review,app_id,name,url,author.steamid,recommended,votes_helpful,votes_funny,weighted_vote_score,...,author.playtime_at_review,author.last_played,written_during_early_access,developer,desc_snippet,game_description,genre,popular_tags,game_details,languages
0,8857737,Very good game and not demanding at all i like...,205100,Dishonored,https://store.steampowered.com/app/205100/Dish...,76561197960270613,True,0,0,0.000000,...,1860.0,1.390720e+09,False,Arkane Studios,Dishonored is an immersive first-person action...,About This Game Dishonored is an immersive fi...,"Action,Adventure","Stealth,First-Person,Action,Steampunk,Assassin...","Single-player,Steam Achievements,Full controll...","English,German,French,Italian,Spanish - Spain"
1,25581518,it will be time to say goodbye to earth soon a...,275850,No Man's Sky,https://store.steampowered.com/app/275850/No_M...,76561197960270613,True,1,0,0.519868,...,718.0,1.563156e+09,False,Hello Games,No Man's Sky is a game about exploration and s...,About This Game Inspired by the adventure and...,"Action,Adventure","Open World,Space,Exploration,Sci-fi,First-Pers...","Single-player,Multi-player,Online Multi-Player...","English,French,Italian,German,Spanish - Spain,..."
2,26131004,Just started playing this masterpiece,292030,The Witcher® 3: Wild Hunt,https://store.steampowered.com/app/292030/The_...,76561197960270613,True,5,0,0.536000,...,456.0,1.499520e+09,False,CD PROJEKT RED,As war rages on throughout the Northern Realms...,About This Game The Witcher: Wild Hunt is a s...,RPG,"Open World,RPG,Story Rich,Atmospheric,Mature,F...","Single-player,Steam Achievements,Full controll...","English,French,Italian,German,Spanish - Spain,..."
3,28354619,Bye Bye Infinite Warfare,292730,Call of Duty®: Infinite Warfare,https://store.steampowered.com/app/292730/Call...,76561197960270613,False,0,0,0.000000,...,3970.0,1.507938e+09,False,Infinity Ward,Infinite Warfare delivers three unique game mo...,About This Game Includes the Terminal Bonus ...,"Action,Adventure","Action,FPS,Multiplayer,Futuristic,Zombies,Spac...","Single-player,Online Multi-Player,Online Co-op...","English,French,Italian,German,Spanish - Spain,..."
4,29066224,its fun and now supports multiplayer just buy it,225540,Just Cause™ 3,https://store.steampowered.com/app/225540/Just...,76561197960270613,True,2,0,0.513423,...,821.0,1.464981e+09,False,Avalanche Studios,With over 1000 km² of complete freedom from sk...,About This Game The Mediterranean republic o...,"Action,Adventure","Open World,Action,Destruction,Third-Person Sho...","Single-player,Steam Achievements,Full controll...","English,French,Italian,German,Spanish - Spain,..."


In [54]:
test_df=df_re.groupby(['author.steamid']).apply(lambda x : x.sort_values(by = 'datetime_created', ascending = True).tail(round(len(x)*0.2))).reset_index(drop = True)

In [55]:
train_set_userids = set(train_df['author.steamid'].unique())
test_set_userids = set(test_df['author.steamid'].unique())
matching_users = train_set_userids.intersection(test_set_userids) 

In [56]:
steamId = train_df['author.steamid'].unique()
len(steamId) 

10525

In [57]:
num_predict = {}
for i in range(len(steamId)):
    num_predict[str(steamId[i])] = len(test_df[test_df['author.steamid'] == steamId[i]].app_id)

### Recommend algorithm

In [58]:
def getGames(sid):
    i = list(steamId).index(sid)
    return train_df[train_df['author.steamid'] == steamId[i]].app_id.values

In [59]:
def game_re(gameid,total,k):
    temp = []
    position = idList.index(gameid)
    index = np.array(sim_result[position]).argsort()[::-1]
    
    value = np.sort(sim_result[position])[::-1]

    for i in range(len(index)):
        if index[i] not in total:
            temp.append((idList[index[i]],value[i]))

    return temp[:k]

In [60]:
from collections import defaultdict
def group_re(user_i,topn):
    temp = []
    total = getGames(steamId[user_i])

    for game in total:
        temp.append(game_re(game,total,topn))

    subtotal = []
    for re in temp:
        for i in re:
            subtotal.append(i[0])

    subtotal = list(set(subtotal))
    subtotal = dict.fromkeys(subtotal,0)

    for re in temp:
        for i in re:
            subtotal[i[0]] = subtotal[i[0]] + i[1]
    
    big = []
    for t in temp:
        big = big + t

    default_dict = defaultdict(int) 

    for user in big:
        default_dict[user[0]] +=1

    final_dict = {}
    for key in list(default_dict.keys()):
        final_dict[key] = [default_dict[key],subtotal[key]/default_dict[key]]

    final_dict = sorted(final_dict.items(),key=lambda x:x[1],reverse=True)
    result = []
    
    for i in final_dict[:topn]:
        result.append(i[0])
    return result

### Make predictions on the test set

In [61]:
predictions = []
for i in range(len(test_set_userids)):
    predictions.append(group_re(i,10))


In [62]:
recommend = pd.DataFrame(matching_users,columns=['user_id'])
recommend['pred'] = predictions
recommend.head(10)

,user_id,pred
0,76561197982318596,"[379430, 238320, 582010, 311690, 620, 264710, ..."
1,76561198119616517,"[582010, 379430, 638970, 312660, 214560, 26471..."
2,76561197998571524,"[379430, 485510, 268500, 638970, 418370, 25249..."
3,76561198080851981,"[312660, 204360, 214560, 379430, 264710, 23832..."
4,76561198119649294,"[582010, 582160, 214560, 620, 379430, 367520, ..."
5,76561198323662866,"[268910, 311690, 582010, 238320, 206190, 44782..."
6,76561198067482644,"[418370, 356190, 485510, 238320, 379430, 31169..."
7,76561198118797332,"[582010, 379430, 238460, 238320, 485510, 46093..."
8,76561198187544599,"[312660, 440900, 252490, 379430, 485510, 21456..."
9,76561198121025570,"[379430, 264710, 214560, 213670, 424840, 35619..."


## 4.Evaluation

In [105]:
def indicators_5(rankedList, testList):
    #rankedLlist: 预测的二维数组
    #testList: test set的二维数组
    #将传入两个数组的推荐人对应上，比如说rankedlist的第一个是user a，那么testList的第一个也要是user a
    Hits_i = 0
    Len_R = 0
    Len_T = len(testList)
    MRR_i = 0
    HR_i = 0
    HR_i2 = 0
    NDCG_i = 0    
    count = 0

    for i in range(len(testList)):
        for j in testList[i]:
            count = count + 1
    
    for i in range(len(rankedList)):
        for j in range(len(rankedList[i])):
            for k in range(len(testList[i])):
                if testList[i][k]==rankedList[i][j]:
                    Hits_i+=1
                    HR_i+=1
                    HR_i2+=1
                    MRR_i+=1/(j+1)   
                    NDCG_i+=1/(math.log2(1+j+1))
                    break
    HR_i/=Len_T
    HR_i2/=count
    NDCG_i/=Hits_i
    print(Hits_i)
    print(f'HR_method1={HR_i}')
    print(f'HR_method2={HR_i2}')
    print(f'NDCG={NDCG_i}')


In [94]:
test_list = []
for i in test_set_userids:
    test_list.append(test_df[test_df['author.steamid'] == i]['app_id'].tolist())

In [106]:
indicators_5(predictions,test_list)

2993
HR_method1=0.2843705463182898
HR_method2=0.07298753871290267
NDCG=0.45389041708731676
